In [ ]:
import warnings
warnings.filterwarnings("ignore")
import streamlit as st
import pandas as pd
import json
import re
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col
from snowflake.snowpark import types as T
from snowflake.snowpark import Row
from snowflake.core import Root
from snowflake.cortex import Complete
session = get_active_session()

## Need to add snowflake.core and snowflake-ml-python to your packages within your notebook


In [ ]:
-- View the list of files in the stage to ensure 6 are present
LIST @HEALTHCARE_DEMO_DB.PUBLIC.HEALTHCARE_DEMO_STAGE PATTERN='.*\.pdf$';

In [ ]:
-- View a transcript
SELECT
  SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    '@HEALTHCARE_DEMO_DB.PUBLIC.HEALTHCARE_DEMO_STAGE',
    'claim_1.pdf',
    OBJECT_CONSTRUCT('mode','Layout')
  ) AS layout;

### 🧾 Extract PDF Transcript Content with `PARSE_DOCUMENT`

We’ll use the [`PARSE_DOCUMENT`](https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex) function to extract the full contents of each transcript PDF.

Snowflake has simplified working with unstructured documents by providing a Cortex AI SQL function that returns the extracted content as a structured JSON object — no external parsing tools required.

This is the **first step** in working with unstructured data, enabling:
- RAG pipelines powered by Cortex Search
- LLM workflows like summarization, translation, or classification
- Structured output extraction from forms and contracts

💡 `PARSE_DOCUMENT` supports two modes:
- **OCR mode** → Best for clean text extraction from scanned documents  
- **LAYOUT mode** → Best for preserving structure like tables, headers, and sections (used here)

We’ll store the extracted content along with customer name and file path in a `PARSED_TRANSCRIPTS` table for future use.

In [ ]:
CREATE OR REPLACE TABLE PARSED_CLAIM_PDFS (
    CLAIM_ID NUMBER,
    RELATIVE_PATH STRING PRIMARY KEY,
    RAW_TEXT VARIANT,
    LOADED_AT TIMESTAMP_TZ DEFAULT CURRENT_TIMESTAMP()
);

INSERT INTO PARSED_CLAIM_PDFS (CLAIM_ID, RELATIVE_PATH, RAW_TEXT)
SELECT 
    TRY_CAST(REGEXP_REPLACE(RELATIVE_PATH, '.*claim_(\\d+)\\.pdf$', '\\1') AS NUMBER) AS CLAIM_ID,
    RELATIVE_PATH,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        '@HEALTHCARE_DEMO_DB.PUBLIC.HEALTHCARE_DEMO_STAGE',
        RELATIVE_PATH,
        OBJECT_CONSTRUCT('mode','Layout')
    ) AS RAW_TEXT
FROM DIRECTORY('@HEALTHCARE_DEMO_DB.PUBLIC.HEALTHCARE_DEMO_STAGE') f
WHERE 
  RELATIVE_PATH LIKE 'claim_%.pdf'
  AND RELATIVE_PATH NOT IN (SELECT RELATIVE_PATH FROM PARSED_CLAIM_PDFS);


### ✂️ Chunk Transcript Text with SPLIT_TEXT_RECURSIVE_CHARACTER
Once we’ve extracted the full layout of each transcript, the next step is to split the long text into smaller overlapping chunks. This is a critical step before embedding or indexing for semantic search.

We’ll use the [`SPLIT_TEXT_RECURSIVE_CHARACTER`](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex) function, which is designed to split long documents intelligently based on a preferred delimiter (e.g., paragraph breaks).

This prepares the data for downstream use with:
- Cortex Search indexing
- Embedding generation
- Summarization or classification with LLMs

💡 We’re using:
- A chunk size of **800 characters**
- An overlap of **150 characters**
- A preferred break on "\n\n" to preserve paragraph structure

In [ ]:
-- Create the chunks table for claims
CREATE OR REPLACE TABLE CLAIM_PDF_CHUNKS (
    CLAIM_ID NUMBER,
    RELATIVE_PATH STRING,
    CHUNK STRING
);

-- Chunk the parsed PDF text using Cortex's split function
INSERT INTO CLAIM_PDF_CHUNKS (CLAIM_ID, RELATIVE_PATH, CHUNK)
WITH text_chunks AS (
    SELECT
        CLAIM_ID,
        RELATIVE_PATH,
        SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
            RAW_TEXT:content::STRING,  -- extract 'content' field
            'markdown',                 -- tokenizer
            800,                        -- chunk size
            100,                        -- overlap
            ARRAY_CONSTRUCT('\n\n')     -- preferred break
        ) AS CHUNKS
    FROM PARSED_CLAIM_PDFS
    WHERE RAW_TEXT:content IS NOT NULL
)
SELECT
    CLAIM_ID,
    RELATIVE_PATH,
    chunk.value::STRING AS CHUNK
FROM text_chunks,
LATERAL FLATTEN(input => CHUNKS) AS chunk;


### 🔍 Make Your Transcripts Searchable with Cortex Search

Now that we've extracted and chunked the content of each transcript, the next step is to make it **searchable** using Snowflake's fully managed retrieval system: **Cortex Search**.

[`CORTEX SEARCH SERVICE`](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview) enables **hybrid retrieval** — combining vector search (semantic meaning) with keyword search (lexical matching) — to deliver **highly accurate results** out of the box.

This capability is a key enabler for Retrieval-Augmented Generation (RAG), allowing you to:
- Ask contextual questions over your documents using SQL or LLM interfaces
- Power chatbots and copilots with document-grounded responses
- Serve precise and explainable enterprise search across large volumes of unstructured content

💡 Powered by `snowflake-arctic-embed-l-v2.0`, a state-of-the-art embedding model optimized for enterprise search.

Once the search service is defined, you can start querying it with natural language — directly inside Snowflake — without needing external pipelines, vector DBs, or hosting infrastructure.

In [ ]:
-- Create a Cortex Search Service over your chunked transcripts table
CREATE OR REPLACE CORTEX SEARCH SERVICE SNOWFLAKE_INTELLIGENCE.AGENTS.CLAIM_PDF_CHUNKS_SEARCH_SERVICE
  ON CHUNK
  ATTRIBUTES CLAIM_ID, RELATIVE_PATH
  WAREHOUSE = SNOWFLAKE_INTELLIGENCE_WH
  TARGET_LAG = '365 days'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
      CLAIM_ID,
      RELATIVE_PATH,
      CHUNK::VARCHAR AS CHUNK
    FROM SNOWFLAKE_INTELLIGENCE.CONFIG.CLAIM_PDF_CHUNKS
  );

### Test your service — Baseline **“one-shot” RAG** (even with a top-tier model)

Select a customer and then run the *BaselineRAGPipeline* cell to perform the **simplest possible Retrieval-Augmented Generation flow**:

1. **Retrieve** the **first** transcript chunk that semantically matches the user’s question.  
2. **Generate** an answer by stuffing that single chunk into the prompt of a **state-of-the-art model** (*`claude-3-5-sonnet`* or *`mistral-large2`*).

---

> Even with a premium LLM you’ll notice it can only repeat whatever facts happen to live in that lone chunk.  
> It frequently replies with **“Based on the limited context provided …”** because:
>
> * The relevant details might sit in a **different** chunk.  
> * We pass **no metadata** (e.g., speaker tags, meeting header) to help the model reason.  
> * We don’t ask it to **extract** or **structure** anything.
>
> You’ll address all of these gaps in the next steps.

In [ ]:
# Pull distinct CLAIM_IDs from your parsed PDF or chunks table
claim_ids = [
    r["CLAIM_ID"]
    for r in session.table("CLAIM_PDF_CHUNKS")
    .select("CLAIM_ID")
    .distinct()
    .collect()
]

# Sort for a clean dropdown
claim_ids = sorted(claim_ids)

# User selects a claim ID to analyze
selected = st.selectbox(
    "Run this cell, then select a claim ID to analyze",
    options=claim_ids,
    key="selected_claim_id"
)

st.success(f"Chosen claim ID: {st.session_state.selected_claim_id}")


In [ ]:
# Create service handle
root = Root(get_active_session())
claim_id = st.session_state.selected_claim_id
svc  = root.databases["SNOWFLAKE_INTELLIGENCE"]\
           .schemas["AGENTS"]\
           .cortex_search_services["CLAIM_PDF_CHUNKS_SEARCH_SERVICE"]  # --> Use your Snowflake Cortex Search Service

# Specify user question
question = f"""
            Provide an explanation of benefits for claim ID {claim_id}.
            Include diagnosis, procedure, amount billed, amount paid, patient responsibility,
            and a short summary of the visit.
            """.strip()

# Get naive retrieval – grab the very first hit (could be just the title page)
hit = svc.search(
        query            = question,
        columns          = ["CLAIM_ID", "CHUNK", "RELATIVE_PATH"],
        limit            = 5,                # look at a few chunks
        search_type      = "embed"           # basic semantic search
     ).results[0]

st.info(f"**File:** {hit['RELATIVE_PATH']}\n\n{hit['CHUNK']}")

# Single-pass generation
response = Complete(
             model  = "claude-3-5-sonnet",  # --> use Anthropic model or Mistral Large model
             prompt = (
                 f"{question}\n\n"
                 "Answer **only** using the context below:\n"
                 "-----\n"
                 f"{hit['CHUNK']}\n"
                 "-----"
             )
           ).strip()

st.info(f"**LLM Response:**\n\n{response}")